# Recommendation Service API Demo

This notebook demonstrates how to interact with the Recommendation Service API:
- Get home page recommendations
- Get recommendations with explainable reasons
- Get similar courses
- Parse and visualize recommendations


In [ ]:
import httpx
import json
from typing import Dict, Any, List
import pandas as pd
import asyncio

# Configuration
BASE_URL = "http://localhost:8002"  # Adjust if your service runs on different port


## 1. Get Home Page Recommendations


In [ ]:
async def get_home_recommendations(
    user_id: str,
    explain: bool = False
) -> List[Dict[str, Any]]:
    """Get home page recommendations for a user."""
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{BASE_URL}/api/v1/recommendations/home",
            params={"user_id": user_id, "explain": explain}
        )
        response.raise_for_status()
        data = response.json()
        
        if explain and "recommendations" in data:
            return data["recommendations"]
        return data

# Get basic recommendations
recommendations = await get_home_recommendations("user123", explain=False)

print(f"Found {len(recommendations)} recommendations:\n")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec['title']}")
    print(f"   Level: {rec['level']}")
    print(f"   Tags: {', '.join(rec.get('tags', []))}")
    print()


## 2. Get Recommendations with Explanations


In [ ]:
# Get recommendations with explainable reasons
recommendations = await get_home_recommendations("user123", explain=True)

print(f"Found {len(recommendations)} recommendations with explanations:\n")
for i, rec in enumerate(recommendations, 1):
    course = rec["course"]
    print(f"{i}. {course['title']}")
    print(f"   Score: {rec.get('score', 'N/A'):.3f}")
    print(f"   Reason: {rec['reason']}")
    print()


## 3. Get Similar Courses


In [ ]:
async def get_similar_courses(course_id: str) -> List[Dict[str, Any]]:
    """Get courses similar to a given course."""
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"{BASE_URL}/api/v1/recommendations/similar/{course_id}"
        )
        response.raise_for_status()
        return response.json()

# Get similar courses
similar = await get_similar_courses("course_python_basic")

print(f"Found {len(similar)} similar courses to 'course_python_basic':\n")
for i, course in enumerate(similar, 1):
    print(f"{i}. {course['title']}")
    print(f"   Level: {course['level']}")
    print(f"   Description: {course['description'][:80]}...")
    print()


## 4. Visualize Recommendations


In [ ]:
# Convert recommendations to DataFrame for easier analysis
recommendations = await get_home_recommendations("user123", explain=True)

if recommendations:
    data = []
    for rec in recommendations:
        course = rec["course"]
        data.append({
            "Title": course["title"],
            "Level": course["level"],
            "Score": rec.get("score", 0.0),
            "Reason": rec["reason"],
            "Tags": ", ".join(course.get("tags", []))
        })
    
    df = pd.DataFrame(data)
    print(df.to_string(index=False))


## 5. Compare Recommendations for Different Users


In [ ]:
# Get recommendations for multiple users
users = ["user1", "user2", "user3"]

all_recommendations = {}
for user_id in users:
    recs = await get_home_recommendations(user_id, explain=False)
    all_recommendations[user_id] = [r["title"] for r in recs]

print("Recommendations by user:\n")
for user_id, titles in all_recommendations.items():
    print(f"{user_id}:")
    for title in titles:
        print(f"  - {title}")
    print()
